In [28]:
import pandas as pd
import numpy as np
import glob
#import nibabel as nib
from scipy.ndimage import label
from numpy.linalg import inv

## msid + mseid data

In [79]:
#Construct data table
data_cols = ["msid",
             "tot_years_gt1_lesion_present",
             "inf_years_gt1_lesion_present",
             "jux_years_gt1_lesion_present",
             "per_years_gt1_lesion_present",
             "sub_years_gt1_lesion_present",
             "tot_lesion_presentation",
             "inf_lesion_presentation",
             "jux_lesion_presentation",
             "per_lesion_presentation",
             "sub_lesion_presentation",
             "comparison",
             "tot_lesion_change_percent",
             "tot_lesion_change_absolute",
             "tot_lesion_vol_change_percent",
             "tot_lesion_vol_change_absolute",
             "inf_lesion_change_percent",
             "inf_lesion_change_absolute",
             "inf_lesion_vol_change_percent",
             "inf_lesion_vol_change_absolute",
             "jux_lesion_change_percent",
             "jux_lesion_change_absolute",
             "jux_lesion_vol_change_percent",
             "jux_lesion_vol_change_absolute",
             "per_lesion_change_percent",
             "per_lesion_change_absolute",
             "per_lesion_vol_change_percent",
             "per_lesion_vol_change_absolute",
             "sub_lesion_change_percent",
             "sub_lesion_change_absolute",
             "sub_lesion_vol_change_percent",
             "sub_lesion_vol_change_absolute",
             "tot_number_increasing_0",
             "tot_number_increasing_10",
             "tot_number_increasing_20",
             "tot_number_increasing_50",
             "tot_number_increasing_100",
             "inf_number_increasing_0",
             "inf_number_increasing_10",
             "inf_number_increasing_20",
             "inf_number_increasing_50",
             "inf_number_increasing_100",
             "jux_number_increasing_0",
             "jux_number_increasing_10",
             "jux_number_increasing_20",
             "jux_number_increasing_50",
             "jux_number_increasing_100",
             "per_number_increasing_0",
             "per_number_increasing_10",
             "per_number_increasing_20",
             "per_number_increasing_50",
             "per_number_increasing_100",
             "sub_number_increasing_0",
             "sub_number_increasing_10",
             "sub_number_increasing_20",
             "sub_number_increasing_50",
             "sub_number_increasing_100",
             "tot_number_decreasing_0",
             "tot_number_decreasing_10",
             "tot_number_decreasing_20",
             "tot_number_decreasing_50",
             "tot_number_decreasing_100",
             "inf_number_decreasing_0",
             "inf_number_decreasing_10",
             "inf_number_decreasing_20",
             "inf_number_decreasing_50",
             "inf_number_decreasing_100",
             "jux_number_decreasing_0",
             "jux_number_decreasing_10",
             "jux_number_decreasing_20",
             "jux_number_decreasing_50",
             "jux_number_decreasing_100",
             "per_number_decreasing_0",
             "per_number_decreasing_10",
             "per_number_decreasing_20",
             "per_number_decreasing_50",
             "per_number_decreasing_100",
             "sub_number_decreasing_0",
             "sub_number_decreasing_10",
             "sub_number_decreasing_20",
             "sub_number_decreasing_50",
             "sub_number_decreasing_100",
             "tot_new_lesions",
             "tot_lost_lesions",
             "inf_new_lesions",
             "inf_lost_lesions",
             "jux_new_lesions",
             "jux_lost_lesions",
             "per_new_lesions",
             "per_lost_lesions",
             "sub_new_lesions",
             "sub_lost_lesions",
             "lesion",
             "in_both",
             "overlap",
             "volume_change",
             "change_in_lesion_type"]
analysis = {}
for item in data_cols:
    analysis[item] = []
prev_data = pd.DataFrame.from_csv('clinical_info.csv')
prev_data

,mse,total_lesions,max_lesion_volume,min_lesion_volume,total_volume_of_lesions,average_volume,median_volume,inf_total_lesions,inf_max_lesion_volume,inf_min_lesion_volume,...,sub_max_lesion_volume,sub_min_lesion_volume,sub_total_volume_of_lesions,sub_average_volume,sub_median_volume,edss,dc,dd,msfc 25 1,msfc 25 2
0,mse1,15,429.0,15.0,1661.0,110.733333,72.0,1,53.0,53.0,...,311.0,15.0,459.0,114.750000,66.5,3.5,RR,35.0,6.3,6.1
1,mse1003,16,4427.0,44.0,9706.0,606.625000,108.5,0,0.0,0.0,...,4427.0,44.0,5092.0,727.428571,108.0,2.0,RR,9.0,4.6,4.7
2,mse1009,6,81.0,40.0,346.0,57.666667,55.5,0,0.0,0.0,...,81.0,43.0,241.0,60.250000,58.5,2.0,RR,15.0,3.8,3.6
3,mse1011,5,2154.0,23.0,2425.0,485.000000,96.0,0,0.0,0.0,...,2154.0,23.0,2329.0,582.250000,76.0,1.0,CIS,3.0,4.3,4.2
4,mse1017,3,123.0,72.0,310.0,103.333333,115.0,0,0.0,0.0,...,72.0,72.0,72.0,72.000000,72.0,0.0,RR,0.0,4.0,4.2
5,mse1023,7,358.0,75.0,1250.0,178.571429,168.0,0,0.0,0.0,...,168.0,75.0,243.0,121.500000,121.5,1.5,RR,5.0,4.3,4.2
6,mse1027,15,3584.0,19.0,10451.0,696.733333,336.0,0,0.0,0.0,...,3584.0,19.0,4031.0,806.200000,176.0,3.0,RR,14.0,4.9,4.7
7,mse1033,18,1988.0,23.0,6286.0,349.222222,169.0,0,0.0,0.0,...,1111.0,23.0,1940.0,242.500000,132.0,4.5,PP,29.0,6.0,5.4
8,mse104,80,18514.0,29.0,55510.0,693.875000,111.5,7,210.0,43.0,...,675.0,29.0,2828.0,148.842105,102.0,3.5,RR,9.0,6.1,5.8
9,mse1045,12,1193.0,20.0,2348.0,195.666667,56.0,0,0.0,0.0,...,243.0,20.0,699.0,87.375000,56.0,4.0,RR,20.0,4.5,4.4


In [78]:
#created matrix of tps
tp_matrix = {}
count = 1
for x in range(1,10):
    for y in range(x+1,11):
        tp_matrix["%s,%s" %(x,y)]=count
        count += 1
tp_matrix

#created test data and dataframe, sorted by mkid and tp
test = {"mkid" : ["mk1","mk2","mk3","mk4","mk1","mk2","mk3","mk4","mk1","mk2","mk3","mk4"],
        "mkeid" : ["mke009","mke002","mke003","mke004","mke005","mke006","mke007","mke008",
                   "mke001","mke010","mke011","mke012"],
        "tp" : [1,1,1,1,2,2,2,3,3,3,5,4],
        "tot_les" : [5,7,10,14,1,4,24,13,25,27,19,4],
       }
testdf = pd.DataFrame(test, columns=["mkid","mkeid","tp","tot_les"]).sort_values(by=['mkid','tp'])
testids = testdf.mkid.values
df = pd.DataFrame(prev_data).sort_values(by=['msid'])
ids = df.msid

#percent change function
def percentchange(num2,num1):
    value = ((float(num2)-float(num1))/(float(num1))) * 100
    return value

#perform calculations
print "mkid", "comparison", "abs_lesion_change", "per_lesion_change"
for x in range(0,len(ids),3):
    val = testdf['mkid']==ids[x] #shortcut to gathering variables by mkid
    tp_list = testdf[val].tp.values 
    mkeid_list = testdf[val].mkeid.values
    tot_les_list = testdf[val].tot_les.values
    tp_current = []
    tp_index_calc = []
    for y in range(len(tp_list)):
        for z in range(y+1,len(tp_list)):
            tp_current.append("%s,%s" % (tp_list[y],tp_list[z]))
            tp_index_calc.append([y,z])
    for y in range(len(tp_current)):
        les_change_abs = tot_les_list[tp_index_calc[y][1]] - tot_les_list[tp_index_calc[y][0]]
        les_change_per = percentchange(tot_les_list[tp_index_calc[y][1]],tot_les_list[tp_index_calc[y][0]])
        print ids[x], tp_matrix[tp_current[y]], les_change_abs, "%.2f%%" % les_change_per
    print
 

mkid comparison abs_lesion_change per_lesion_change
mk1 1 -4 -80.00%
mk1 2 20 400.00%
mk1 10 24 2400.00%

mk2 1 -3 -42.86%
mk2 2 20 285.71%
mk2 10 23 575.00%

mk3 1 14 140.00%
mk3 4 9 90.00%
mk3 12 -5 -20.83%

mk4 2 -1 -7.14%
mk4 3 -10 -71.43%
mk4 18 -9 -69.23%

